本文内容主要来自：
- https://arxiv.org/pdf/1710.11041.pdf

## 摘要
尽管最近NMT取得了不错的成绩，但是其训练数据（对应双语文集）的获取，对于一些语言来说，一直都是一个问题。根据这种情况，我们提出一种无监督的学习方案。这种方案只需要用到单语言文集。这个模型主要是建立在unsupervised embedding mappings之上，并且引入了经过部分修改的基于Attention机制的encoder-decoder模型。

## 介绍
这里提出的系统是建立在公共encoder的基础上的。这个公共encoder适用于双向转换，并且使用前面提到cross-ligual embedding。多亏了这个公共的encoder，才使得我们可以通过单一的语言数据集来完成整个系统的训练。为了能够学到有用的结构信息，我们通过随机改变输入中的符号来添加噪音。除此之外，我们还在训练过程汇总添加backtranslation来优化结果。如下图，描述了模型的结构。
![unsupervised NMT](https://raw.githubusercontent.com/HuangYiran/readPaper/master/fotos/unsupervised_NMT.png)


## 系统结构
我们提出的这个系统和一般的系统不同的地方在于：
- Dual Structure，一般的NMT系统的都是针对特定方向的翻译设定的，比如英译德，德译英。但我们这是使用的是双向的翻译系统。
- shared encoder，在我们的系统中，针对进行互译的语言中不同的语言，我们共用一个encoder。这有点像Ha et al.,(2016), Lee et al.(2017), Johnson et al.,(2017)。使用这个通用的encoder的目的在于，将输入以不依赖与具体的语言的方式进行表达。
- Fixed embeddings in the encoder, 大部分的NMT系统会随机初始化embedding，然后在训练的过程中，对其进行调整。与之相反，我们在encoder中使用事先训练好的cross-lingual embeddings。并在训练的过程中，保持这些embedding不变。通过这种方式，encoder获得了，不依赖于具体语言的，word-level的表达能力。现在只需要考虑如果更进一步结合这些单个的单词，使得能够表达比较大的短语。

## Unsupervised Training
使用上面提到的系统，结合下面两个策略，我们实现这个系统的依赖于单独语库的非监督学习：
- Denosing，对于给定的输入语句，因为我们的encoder使用的是fixed cross-lingual embeddings，所以无论输入的是哪种语言，我们都可以把他转化为对应的不依赖于具体语言的表达方式，然后我们通过对应输入语言的decoder，尝试还原输入。<br> 
然而这种方法可能存在的一个漏洞是，这个网络其实只是实现了复制的功能（像Autoencoder），而没有学习输入语言的内部结构。<br>
为了避免出现这种情况，我们建议在输入中使用随机噪音。为了实现这个目标，我们随机替换输入句子中单词的顺序。举个例子，我们输入一个包含有N个元素的句子，我们随机执行N/2次交换元素的操作。通过这种方法，系统就必须学习单词之间的结构信息，从而重新确定这些单词的排列顺序。同时，通过降低系统对输入单词顺序的依赖，使得系统可以更好的理解学习不同语言之间语序的不同之处。
- Backtranslation，尽管使用了Denosing，上面的训练过程也不过是一个做了一些调整的复制的过程，最重要的是，他每次只使用单一的怨言，他并没有实现我们的真正的目的，也就是不同语言之间的互相翻译。为了在没有违背使用单词语料集的诺言的前提下实现我们的目标。我们使用了由Sennrich et al.提出的backtranslation的方法。具体的说就是给定一种语言的输入，我们使用系统在reference模式下，结合greedy方法，通过decoder翻译为另外一种语言。通过这种方法来得到伪并行语料集。并以此来训练系统。

在训练的过程中，我们交替的使用上面提到的两个过程。假设我们有两个语言L1和L2，在一个周期中，我们先对L1做一个Denosing，然后也给L2来一个。之后再从L1到L2做backtranslation，然后再反过来做一次。

## 可能有趣的paper
- backtranslation(Sennrich et al., 2016a)
- denoising(Vincent et al., 2010, Hill et al., 2017)
- shared encoder Ha et al.,(2016), Lee et al.(2017), Johnson et al.,(2017)
- Dual nature of machine translation(He et al., 2016)
- unsupervised cross-lingual embeddings(Artetxe et al., 2017; Zhang et al., 2017)
- pivoting or triangulation techniques(Chen et al., 2017)
- semi-supervised approaches(He et al., 2016)

## 可能有趣的其他东西
- 论文中2.0相关的内容还是要看一下的，感觉也是相当有趣的。